# NSAI Level 1B: Multi-Detector Neuro-Symbolic Intent Classification

**Architecture Shift:**
- **Level 1A**: Single multi-class classifier
- **Level 1B**: One binary detector per intent

**Core Concept:**
- Each detector independently scores: "Does this match MY intent?"
- Scores ∈ [0,1], do NOT sum to 1
- Rules (not models) decide final outcome

**Dataset**: utterance, intent (4 classes: investigate, execution, summarization, out_of_scope)

**Output**: Deterministic, explainable decisions with detector scores + rule governance

**Method**: Multi-detector statistical learning + symbolic rule precedence (no LLMs, no timestamps)

## Part 1: Data Preparation

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Import Level 1B module
from level1b_model import Level1BClassifier, RULE_PRIORITY, RULE_TYPES, get_configuration


In [2]:
# Load & Validate Data
df = pd.read_csv('../data/intents_base.csv')

# Assert columns
assert set(df.columns) == {'utterance', 'intent'}, f"Expected columns {{utterance, intent}}, got {set(df.columns)}"

# Normalize intent labels
df['intent'] = df['intent'].str.lower().str.strip()

# Show class distribution
print("Class Distribution:")
print(df['intent'].value_counts())
print(f"\nTotal: {len(df)} records")

Class Distribution:
intent
out_of_scope     169
execution        150
investigate      149
summarization    146
Name: count, dtype: int64

Total: 614 records


In [12]:
# Train/Test Split
X = df['utterance']
y = df['intent']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

Train: 491 | Test: 123


## Part 2: Multi-Detector Training

In [13]:
# Intent-Specific Detector Models
# Train ONE binary classifier per intent

intents = ['investigate', 'execution', 'summarization', 'out_of_scope']
detectors = {}

print("Training binary detectors...\n")

for intent in intents:
    # Create binary labels: 1 if utterance matches this intent, 0 otherwise
    y_binary_train = (y_train == intent).astype(int)
    y_binary_test = (y_test == intent).astype(int)
    
    # Create detector pipeline
    detector = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 2),
            stop_words='english'
        )),
        ('classifier', LogisticRegression(
            solver='lbfgs',
            random_state=42,
            max_iter=1000
        ))
    ])
    
    # Train detector
    detector.fit(X_train, y_binary_train)
    
    # Evaluate
    y_pred = detector.predict(X_test)
    report = classification_report(y_binary_test, y_pred, output_dict=True)
    
    # Store detector
    detectors[intent] = detector
    
    print(f"{intent:>20} detector: Accuracy={report['accuracy']:.4f}, F1={report['1']['f1-score']:.4f}")

print(f"\nAll {len(detectors)} detectors trained")

Training binary detectors...

         investigate detector: Accuracy=0.8130, F1=0.3784
           execution detector: Accuracy=0.8211, F1=0.4211
       summarization detector: Accuracy=0.9512, F1=0.8846
        out_of_scope detector: Accuracy=0.8130, F1=0.4889

All 4 detectors trained


## Part 4: Level 1B Symbolic Rules

In [ ]:
# Level 1B Rule Configuration (from module)
config = get_configuration()

print("Level 1B Configuration:")
print(f"  BASE_MIN_SCORE: {config['BASE_MIN_SCORE']}")
print(f"  AMBIGUITY_MARGIN: {config['AMBIGUITY_MARGIN']}")
print(f"  EXECUTION_MIN_SCORE: {config['EXECUTION_MIN_SCORE']}")
print(f"  MIN_TOKENS_OUT_OF_SCOPE: {config['MIN_TOKENS_OUT_OF_SCOPE']}")
print(f"  CONCURRENCE_THRESHOLD: {config['CONCURRENCE_THRESHOLD']}")
print(f"\nRule Priority: {config['RULE_PRIORITY']}")
print(f"\nRule Types:")
for rule, rtype in config['RULE_TYPES'].items():
    print(f"  {rule:>30} → {rtype}")


Level 1B Configuration:
  BASE_MIN_SCORE: 0.5
  AMBIGUITY_MARGIN: 0.1
  EXECUTION_MIN_SCORE: 0.85
  MIN_TOKENS_OUT_OF_SCOPE: 3
  CONCURRENCE_THRESHOLD: 0.7

Rule Priority: ['R_LOW_TOKEN_COUNT', 'R_NO_CONFIDENT_DETECTOR', 'R_EXEC_SAFETY', 'R_MULTI_DETECTOR_CONCURRENCE', 'R_AMBIGUOUS', 'R_DEFAULT']

Rule Types:
               R_LOW_TOKEN_COUNT → quality
         R_NO_CONFIDENT_DETECTOR → quality
                   R_EXEC_SAFETY → safety
    R_MULTI_DETECTOR_CONCURRENCE → compound_intent
                     R_AMBIGUOUS → ambiguity
                       R_DEFAULT → default


In [ ]:
# Initialize Level 1B Classifier with trained detectors
classifier = Level1BClassifier(detectors=detectors, intents=intents)

print("Level 1B classifier initialized with multi-detector architecture")
print("✓ Gap 1 fixed: Explicit R_NO_CONFIDENT_DETECTOR rule")
print("✓ Gap 2 fixed: All rules tagged with rule_type")
print("✓ Gap 3 fixed: R_MULTI_DETECTOR_CONCURRENCE reserved for L2")


Level 1B symbolic rules defined
✓ Gap 1 fixed: Explicit R_NO_CONFIDENT_DETECTOR rule
✓ Gap 2 fixed: All rules tagged with rule_type
✓ Gap 3 fixed: R_MULTI_DETECTOR_CONCURRENCE reserved for L2


## Part 6: Explainable Demo

In [ ]:
# Explainable Demo Output
test_utterances = [
    "why is server cpu high",
    "restart nginx on host123",
    "summarize the incident from yesterday",
    "hello",
    "delete all production databases",
    "server issues yesterday"
]

print("="*100)
print("LEVEL 1B: MULTI-DETECTOR NEURO-SYMBOLIC INFERENCE")
print("="*100)

for utterance in test_utterances:
    result = classifier.predict(utterance)
    
    print(f"\n{'-'*100}")
    print(f"UTTERANCE: {utterance}")
    print(f"{'-'*100}")
    
    # Detector scores
    print(f"\nDETECTOR SCORES:")
    for intent, score in sorted(result['detector_scores'].items(), key=lambda x: x[1], reverse=True):
        bar = '█' * int(score * 50)
        print(f"  {intent:>20}: {score:.4f} {bar}")
    
    print(f"\n  Top Detector: {result['top_detector']}")
    print(f"  Score Margin: {result['score_margin']:.4f}")
    print(f"  Meaningful Tokens: {result['meaningful_tokens']}")
    
    # Triggered rules with rule_type
    print(f"\nTRIGGERED RULES:")
    for rule in sorted(result['triggered_rules'], key=lambda r: r['priority'], reverse=True):
        rule_type = rule.get('rule_type', 'unknown')
        print(f"  [{rule['priority']:3d}] {rule['rule_id']:30s} (type: {rule_type})")
        print(f"       Condition: {rule['condition']}")
        print(f"       Signal Value: {rule['value']}")
        if 'explanation' in rule:
            print(f"       → {rule['explanation']}")
    
    # Final decision
    print(f"\nFINAL DECISION:")
    print(f"  Predicted Intent: {result['predicted_intent']}")
    print(f"  Decision State: {result['decision_state']}")
    print(f"  Decision Reason: {result['decision_reason']}")
    
    # Structured JSON
    print(f"\nSTRUCTURED OUTPUT:")
    output_json = {
        'utterance': utterance,
        'predicted_intent': result['predicted_intent'],
        'decision_state': result['decision_state'],
        'decision_reason': result['decision_reason'],
        'detector_scores': result['detector_scores'],
        'score_margin': result['score_margin'],
        'meaningful_tokens': result['meaningful_tokens']
    }
    print(json.dumps(output_json, indent=2))

print(f"\n{'='*100}")
print("DEMO COMPLETE")
print("="*100)


LEVEL 1B: MULTI-DETECTOR NEURO-SYMBOLIC INFERENCE

----------------------------------------------------------------------------------------------------
UTTERANCE: why is server cpu high
----------------------------------------------------------------------------------------------------

DETECTOR SCORES:
           investigate: 0.4892 ████████████████████████
             execution: 0.2212 ███████████
          out_of_scope: 0.1736 ████████
         summarization: 0.1229 ██████

  Top Detector: investigate
  Score Margin: 0.2681
  Meaningful Tokens: 4

TRIGGERED RULES:
  [ 95] R_NO_CONFIDENT_DETECTOR        (type: quality)
       Condition: all detector_scores < 0.5
       Signal Value: 0.4892277767198669
       → No detector confident enough to classify

FINAL DECISION:
  Predicted Intent: out_of_scope
  Decision State: blocked
  Decision Reason: R_NO_CONFIDENT_DETECTOR

STRUCTURED OUTPUT:
{
  "utterance": "why is server cpu high",
  "predicted_intent": "out_of_scope",
  "decision_stat

## Validation: Test Set Evaluation

In [ ]:
# Evaluate Level 1B on test set
print("Evaluating Level 1B on test set...\n")

predictions = []
decision_states = []
rule_counts = {rule: 0 for rule in RULE_PRIORITY}

for text, true_intent in zip(X_test, y_test):
    result = classifier.predict(text)
    predictions.append(result['predicted_intent'])
    decision_states.append(result['decision_state'])
    
    # Count triggered rules
    for rule in result['triggered_rules']:
        rule_counts[rule['rule_id']] += 1

# Accuracy
accuracy = sum(p == t for p, t in zip(predictions, y_test)) / len(y_test)
print(f"Accuracy: {accuracy:.4f}")

# Decision state distribution
print(f"\nDecision State Distribution:")
from collections import Counter
state_counts = Counter(decision_states)
for state, count in state_counts.items():
    print(f"  {state:>20}: {count:3d} ({count/len(decision_states)*100:.1f}%)")

# Rule trigger counts
print(f"\nRule Trigger Counts:")
for rule in RULE_PRIORITY:
    count = rule_counts[rule]
    print(f"  {rule:>20}: {count:3d} ({count/len(X_test)*100:.1f}%)")

# Full classification report
print(f"\nClassification Report:")
print(classification_report(y_test, predictions))


Evaluating Level 1B on test set...

Accuracy: 0.4878

Decision State Distribution:
               blocked:  97 (78.9%)
              accepted:  22 (17.9%)
   needs_clarification:   4 (3.3%)

Rule Trigger Counts:
     R_LOW_TOKEN_COUNT:  52 (42.3%)
  R_NO_CONFIDENT_DETECTOR:  45 (36.6%)
         R_EXEC_SAFETY:   4 (3.3%)
  R_MULTI_DETECTOR_CONCURRENCE:   0 (0.0%)
           R_AMBIGUOUS:   0 (0.0%)
             R_DEFAULT:  22 (17.9%)

Classification Report:
               precision    recall  f1-score   support

    execution       1.00      0.13      0.24        30
  investigate       1.00      0.17      0.29        30
 out_of_scope       0.35      1.00      0.52        34
summarization       1.00      0.59      0.74        29

     accuracy                           0.49       123
    macro avg       0.84      0.47      0.44       123
 weighted avg       0.82      0.49      0.44       123



In [ ]:
# Save Level 1B Results to Artifacts
import os
from datetime import datetime

# Create artifacts directory
artifacts_dir = 'artifacts/level1b'
os.makedirs(artifacts_dir, exist_ok=True)

# 1. Save the trained model
print("Saving trained model...")
model_dir = 'models/level1b'
classifier.save(model_dir)

# 2. Save detailed predictions with all metadata
print("\nSaving detailed predictions...")
detailed_results = []

for i, (text, true_intent) in enumerate(zip(X_test, y_test)):
    result = classifier.predict(text)
    
    detailed_results.append({
        'test_index': i,
        'utterance': text,
        'true_intent': true_intent,
        'predicted_intent': result['predicted_intent'],
        'decision_state': result['decision_state'],
        'decision_reason': result['decision_reason'],
        'top_detector': result['top_detector'],
        'score_margin': result['score_margin'],
        'meaningful_tokens': result['meaningful_tokens'],
        **{f'score_{intent}': result['detector_scores'][intent] for intent in intents},
        'triggered_rules': [r['rule_id'] for r in result['triggered_rules']],
        'primary_rule_type': result['triggered_rules'][-1].get('rule_type', 'unknown') if result['triggered_rules'] else 'none'
    })

# Save as CSV
df_results = pd.DataFrame(detailed_results)
csv_path = f"{artifacts_dir}/level1b_predictions.csv"
df_results.to_csv(csv_path, index=False)
print(f"✓ Saved predictions: {csv_path}")

# Save as JSONL for debugging
jsonl_path = f"{artifacts_dir}/level1b_predictions.jsonl"
with open(jsonl_path, 'w') as f:
    for result in detailed_results:
        f.write(json.dumps(result) + '\n')
print(f"✓ Saved JSONL: {jsonl_path}")

# 3. Save evaluation metrics
print("\nSaving evaluation metrics...")
config = get_configuration()
eval_metrics = {
    'model': 'level1b_multi_detector',
    'timestamp': datetime.now().isoformat(),
    'test_size': len(X_test),
    'accuracy': accuracy,
    'decision_state_distribution': dict(state_counts),
    'rule_trigger_counts': rule_counts,
    'rule_trigger_percentages': {rule: count/len(X_test)*100 for rule, count in rule_counts.items()},
    'classification_report': classification_report(y_test, predictions, output_dict=True),
    'configuration': config
}

metrics_path = f"{artifacts_dir}/evaluation_metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(eval_metrics, f, indent=2)
print(f"✓ Saved metrics: {metrics_path}")

# 4. Save rule activation analysis
print("\nSaving rule activation analysis...")
rule_analysis = []
for rule in RULE_PRIORITY:
    rule_results = [r for r in detailed_results if rule in r['triggered_rules']]
    
    rule_analysis.append({
        'rule_id': rule,
        'rule_type': RULE_TYPES.get(rule, 'unknown'),
        'trigger_count': rule_counts[rule],
        'trigger_percentage': rule_counts[rule]/len(X_test)*100,
        'decision_states': dict(Counter([r['decision_state'] for r in rule_results]))
    })

df_rules = pd.DataFrame(rule_analysis)
rules_path = f"{artifacts_dir}/rule_activation_stats.csv"
df_rules.to_csv(rules_path, index=False)
print(f"✓ Saved rule stats: {rules_path}")

# 5. Save decision state breakdown by intent
print("\nSaving decision state breakdown...")
state_breakdown = []
for intent in intents:
    intent_predictions = [r for r in detailed_results if r['predicted_intent'] == intent]
    
    state_breakdown.append({
        'predicted_intent': intent,
        'total_predictions': len(intent_predictions),
        'accepted': sum(1 for r in intent_predictions if r['decision_state'] == 'accepted'),
        'needs_clarification': sum(1 for r in intent_predictions if r['decision_state'] == 'needs_clarification'),
        'blocked': sum(1 for r in intent_predictions if r['decision_state'] == 'blocked'),
        'avg_score_margin': np.mean([r['score_margin'] for r in intent_predictions]) if intent_predictions else 0
    })

df_states = pd.DataFrame(state_breakdown)
states_path = f"{artifacts_dir}/decision_state_breakdown.csv"
df_states.to_csv(states_path, index=False)
print(f"✓ Saved state breakdown: {states_path}")

print(f"\n{'='*80}")
print(f"All Level 1B artifacts saved")
print(f"{'='*80}")
print(f"Model: {model_dir}/")
print(f"Artifacts: {artifacts_dir}/")
print(f"\nFiles created:")
print(f"  - level1b_predictions.csv (detailed predictions)")
print(f"  - level1b_predictions.jsonl (for debugging)")
print(f"  - evaluation_metrics.json (comprehensive metrics)")
print(f"  - rule_activation_stats.csv (rule analysis)")
print(f"  - decision_state_breakdown.csv (state distribution)")


Saving detailed predictions...
✓ Saved predictions: artifacts/level1b/level1b_predictions.csv
✓ Saved JSONL: artifacts/level1b/level1b_predictions.jsonl

Saving evaluation metrics...
✓ Saved metrics: artifacts/level1b/evaluation_metrics.json

Saving rule activation analysis...
✓ Saved rule stats: artifacts/level1b/rule_activation_stats.csv

Saving decision state breakdown...
✓ Saved state breakdown: artifacts/level1b/decision_state_breakdown.csv

All Level 1B artifacts saved to: artifacts/level1b/
Files created:
  - level1b_predictions.csv (detailed predictions)
  - level1b_predictions.jsonl (for debugging)
  - evaluation_metrics.json (comprehensive metrics)
  - rule_activation_stats.csv (rule analysis)
  - decision_state_breakdown.csv (state distribution)
